In [1]:
import pandas as pd

In [3]:
# 1、获取数据
data = pd.read_csv("../../../../Machine_Learning/Python3天快速入门机器学项目资料/机器学xiday2资料/02-代码/FBlocation/train.csv")

In [4]:
data.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [28]:
# 2、基本的数据处理
# 1）缩小数据范围
data = data.query("x < 2.5 & x > 2 & y < 1.5 & y > 1.0")

In [27]:
data.head()

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
0,0,0.7941,9.0809,54,470702,8523065625,6,1,10
1,1,5.9567,4.7968,13,186555,1757726713,3,5,3
2,2,8.3078,7.0407,74,322648,1137537235,4,6,17
3,3,7.3665,2.5165,65,704587,6567393236,9,4,3
4,4,4.0961,1.1307,31,472130,7440663949,6,1,11


In [29]:
# 2）处理时间特征
time_value = pd.to_datetime(data["time"], unit="s")

In [30]:
date = pd.DatetimeIndex(time_value)

In [31]:
data["day"] = date.day

In [32]:
data["weekday"] = date.weekday

In [33]:
data["hour"] = date.hour

In [11]:
data.head()

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
0,0,0.7941,9.0809,54,470702,8523065625,6,1,10
1,1,5.9567,4.7968,13,186555,1757726713,3,5,3
2,2,8.3078,7.0407,74,322648,1137537235,4,6,17
3,3,7.3665,2.5165,65,704587,6567393236,9,4,3
4,4,4.0961,1.1307,31,472130,7440663949,6,1,11


In [34]:
# 3）过滤签到次数少的地点
place_count = data.groupby("place_id").count()["row_id"]

In [35]:
data.groupby("place_id").count().head()

,row_id,x,y,accuracy,time,day,weekday,hour
place_id,,,,,,,,
1012165853,1,1,1,1,1,1,1,1
1013991737,3,3,3,3,3,3,3,3
1014605271,28,28,28,28,28,28,28,28
1015645743,4,4,4,4,4,4,4,4
1017236154,31,31,31,31,31,31,31,31


In [36]:
place_count[place_count > 3].head()

place_id
1014605271    28
1015645743     4
1017236154    31
1024951487     5
1028119817     4
Name: row_id, dtype: int64

In [37]:
data_final = data[data["place_id"].isin(place_count[place_count > 3].index.values)]

In [38]:
data_final.head()

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
112,112,2.2360,1.3655,66,623174,7663031065,8,3,5
367,367,2.4108,1.3213,74,579667,6644108708,7,2,17
874,874,2.0822,1.1973,320,143566,3229876087,2,4,15
1022,1022,2.0160,1.1659,65,207993,3244363975,3,5,9
1045,1045,2.3859,1.1660,498,503378,6438240873,6,1,19


In [39]:
# 筛选特征值和目标值
x = data_final[["x", "y", "accuracy", "day", "weekday", "hour"]]
y = data_final["place_id"]

In [19]:
x.head()

,x,y,accuracy,day,weekday,hour
0,0.7941,9.0809,54,6,1,10
1,5.9567,4.7968,13,3,5,3
2,8.3078,7.0407,74,4,6,17
3,7.3665,2.5165,65,9,4,3
4,4.0961,1.1307,31,6,1,11


In [41]:
y.head()

112     7663031065
367     6644108708
874     3229876087
1022    3244363975
1045    6438240873
Name: place_id, dtype: int64

In [42]:
# 数据集划分
from sklearn.model_selection import train_test_split

In [43]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [44]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [45]:
# 3）特征工程：标准化
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

# 4）KNN算法预估器
estimator = KNeighborsClassifier()

# 加入网格搜索与交叉验证
# 参数准备
param_dict = {"n_neighbors": [3, 5, 7, 9]}
estimator = GridSearchCV(estimator, param_grid=param_dict, cv=3)
estimator.fit(x_train, y_train)

# 5）模型评估
# 方法1：直接比对真实值和预测值
y_predict = estimator.predict(x_test)
print("y_predict:\n", y_predict)
print("直接比对真实值和预测值:\n", y_test == y_predict)

# 方法2：计算准确率
score = estimator.score(x_test, y_test)
print("准确率为：\n", score)

# 最佳参数：best_params_
print("最佳参数：\n", estimator.best_params_)
# 最佳结果：best_score_
print("最佳结果：\n", estimator.best_score_)
# 最佳估计器：best_estimator_
print("最佳估计器:\n", estimator.best_estimator_)
# 交叉验证结果：cv_results_
print("交叉验证结果:\n", estimator.cv_results_)

D:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


y_predict:
 [9764078387 1732563460 5442484608 ... 7240847236 4861093827 8032468532]
直接比对真实值和预测值:
 4048913     False
11353868    False
1157698     False
8681225     False
1089785     False
            ...  
27680550     True
13219421     True
28245049     True
5470117      True
22868741    False
Name: place_id, Length: 20228, dtype: bool
准确率为：
 0.36543405180937316
最佳参数：
 {'n_neighbors': 7}
最佳结果：
 0.3340199783150061
最佳估计器:
 KNeighborsClassifier(n_neighbors=7)
交叉验证结果:
 {'mean_fit_time': array([0.05365618, 0.05150882, 0.05758015, 0.05224625]), 'std_fit_time': array([0.00839674, 0.00136554, 0.00175555, 0.0007414 ]), 'mean_score_time': array([0.78747869, 0.84149837, 0.9497811 , 0.94546445]), 'std_score_time': array([0.04811002, 0.03762782, 0.0324112 , 0.02087571]), 'param_n_neighbors': masked_array(data=[3, 5, 7, 9],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 7}, {'n_neig